In [1]:
import gradio as gr
import re
import streamlit as st
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool

from docx import Document
from io import BytesIO
import base64

import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] =os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

In [2]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0513')

In [3]:
df=pd.read_csv('./diag_5mcids.csv')
df

,mcid,dos,codeIndex,cd_value,ICD_FULL_DESC,type
0,0,2017-02-28,13415,M25551,Pain in right hip,diag
1,0,2017-06-27,13415,M25551,Pain in right hip,diag
2,0,2017-08-22,13415,M25551,Pain in right hip,diag
3,0,2018-02-22,13415,M25551,Pain in right hip,diag
4,0,2018-04-18,13415,M25551,Pain in right hip,diag
...,...,...,...,...,...,...
816,4,2018-11-01,14174,M546,Pain in thoracic spine,diag
817,4,2019-01-11,14174,M546,Pain in thoracic spine,diag
818,4,2019-05-07,14174,M546,Pain in thoracic spine,diag
819,4,2018-11-29,9753,J309,"Allergic rhinitis, unspecified",diag


In [4]:
df_json = df.to_json()
import json


In [5]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [6]:
@tool
def patientRecordSelect_tool(df_list, mcid):
    """Select the patient's medical records for patient with given mcid.
    'dos': date of service.
    'cd_value': ICD10 diagnosis codes.
    'ICD_FULL_DESC': description of ICD10 code.
    """
    df = pd.DataFrame(df_list)
    output = df[df['mcid'] == mcid][['dos', 'cd_value', 'ICD_FULL_DESC']]
    
    
    markdown_table = "| Date of Service | ICD10 Code | Description |\n"
    markdown_table += "|-----------------|------------|-------------|\n"
    for _, row in output.iterrows():
        markdown_table += f"| {row['dos']} | {row['cd_value']} | {row['ICD_FULL_DESC']} |\n"
    
    # Save the markdown table to a file
    with open("patientRecords.md", "w") as f:
        f.write(f"# Patient Records for MCID {mcid}\n\n")
        f.write(markdown_table)
    
    return markdown_table
   
patientRecordSelect_agent = Agent(
    role="Get patient's medical records",
    goal="For a given patient identified by mcid, get this patient's medical recordes "
         "from a given data using tool patientRecordSelect_tool.",
    backstory="You use given mcid to find the corresponding medical records.",
    tools=[patientRecordSelect_tool],
    allow_delegation=False,
    llm=llm  
)



summary_agent = Agent(
    role="Summarizer",
    goal="Summarize patient's medical records. Find out what diseases this patient had or having, and possible diseases may happen soon. ",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = llm,
    backstory=(
        "As a medical doctor, you have a new patient who brings to you his/her past medical records."
        "Through analyzing this patient's medical records, you find what diseases this patient had in the past, "
        "what chronic desease this patient are having, "
        "and what desease this patient may have in the near future."
    )
)



commonDisease_agent = Agent(
    role="Detector of common diseases",
    goal="Find out if the given patient has diabetes and chronic kidney disease and list the relevant diagnosis codes.",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = llm,
    backstory=(
       "Diabetes and chronic kidney disease are very common, especially among older people. "
        "We are dealing with a Medicare population. Analyze a patient's medical records to see if this patient has these diseases. "
        "If the patient has any of the mentioned diseases, identify the specific ICD-10 codes that indicate these conditions. "
        "Then, suggest a treatment plan for each detected disease."
    )
)



In [7]:
patientRecordSelect_task = Task(
    description="Get the medical records for the given patient {mcid} from file {df_list}. These medical records will be used in later steps",
    expected_output="a table with columns: date of service, ICD10 diagnosis codes, code description",
    agent=patientRecordSelect_agent,
    output_file="patientRecords.md",
    callback=lambda output: save_agent_result("Patient Records", output)
)

summary_task = Task(
    description=(
        "1. Analyze the patient's medical records.\n"
        "2. Provide a summary of past, present, and possible future diseases based on the provided information."
        "3. Limit the length to 200 words."
    ),
    expected_output="A summary of patient's medical conditions.",
    agent=summary_agent,
    context=[patientRecordSelect_task],
    output_file="summary.md",
    callback=lambda output: save_agent_result("Summary", output)
)

commonDisease_task = Task(
    description=(
        "1. Analyze the patient's medical records.\n"
        "2. Answer yes or no for diabetes and chronic kidney disease."
        "3. If the answer is yes for either disease:\n"
        "   a. List the specific ICD-10 codes from the patient's records that indicate the disease.\n"
        "   b. Suggest a treatment plan and dietary restrictions. When searching the internet, make sure to check out the Mayo Clinic website.\n"
        "4. Limit the total response length to 250 words."
    ),
    expected_output="Yes or no answer to common chronic diseases, relevant ICD-10 codes if present, and suggestions if the answer is yes.",
    agent=commonDisease_agent,
    context=[patientRecordSelect_task, summary_task],
    output_file="commonDisease.md",
    callback=lambda output: save_agent_result("Common Diseases", output)
)

In [8]:
chatbot_agent = Agent(
    role="Medical Information Chatbot",
    goal="Provide accurate and helpful responses to user questions about the patient's medical condition or general medical queries",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "You are an AI-powered medical chatbot with the ability to analyze and interpret provided medical records, "
        "summaries, and assessments. You can answer questions "
        "specific to a patient as well as general medical queries. You have access to the outputs of "
        "previous analyses including patient record selection, medical summary, and common disease assessment. "
        "You can also search for additional information when needed to provide comprehensive answers."
    )
)

In [9]:
chatbot_task = Task(
    description=(
        "As a medical chatbot, your task is to answer '{user_question}' asked by the user accurately and helpfully. Follow these steps:\n"
        "1. Review the following patient-specific information:\n"
        "   a. Patient record selection results\n"
        "   b. Medical summary\n"
        "   c. Common disease assessment\n"
        "2. For the user question:\n"
        "   a. First, attempt to answer using the patient-specific information provided.\n"
        "   b. If the patient-specific information is insufficient, then use your general medical knowledge or perform a web search.\n"
        "   c. Provide a clear, concise, and accurate answer, prioritizing patient-specific information when available.\n"
        "   d. Ensure your response is easy for a non-medical professional to understand.\n"
        "   e. If the question is outside your scope or requires immediate medical attention, advise the user to consult a healthcare professional.\n"
        "   f. If you need to make any assumptions or if there's any ambiguity, clearly state this in your response."
    ),
    expected_output="A clear, informative, and accurate response to the user's question, prioritizing patient-specific information when available.",
    agent=chatbot_agent,
    output_file="response.md",
    context=[patientRecordSelect_task, summary_task, commonDisease_task]
)

In [10]:
analysis_crew = Crew(
    agents=[patientRecordSelect_agent, summary_agent, commonDisease_agent],
    tasks=[patientRecordSelect_task, summary_task, commonDisease_task],
    verbose=1
)

# Create separate Crew for chatbot
chatbot_crew = Crew(
    agents=[chatbot_agent],
    tasks=[chatbot_task],
    verbose=1
)

2024-08-06 19:01:05,042 - 139776514393920 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [11]:
results_store={}
import re

import os

def save_agent_result(agent_name, result):
    result_str = str(result)
    print(result_str)

    
    output_files = {
        "Patient Records": "patientRecords.md",
        "Summary": "summary.md",
        "Common Diseases": "commonDisease.md",
        "Medical Information Chatbot": "response.md"
    }

    # If the agent has an associated output file, read its content
    if agent_name in output_files:
        file_path = output_files[agent_name]
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                file_content = file.read()
            results_store[agent_name] = file_content
        else:
            print(f"Warning: Output file {file_path} not found for {agent_name}")
            results_store[agent_name] = result_str
    else:
        # For any agents without output files, store the result as before
        match = re.search(r"exported_output='(.*?)'(?=\s+agent=|\Z)", result_str, re.DOTALL)
        if match:
            exported_output = match.group(1).strip().strip("'")
            results_store[agent_name] = exported_output
        else:
            results_store[agent_name] = result_str

    print(f"Stored result for {agent_name}: {results_store[agent_name][:100]}...")  # Print first 100 chars 

In [12]:
import gradio as gr

In [13]:
def get_agent_result(agent_name):
    return results_store.get(agent_name, "No result available")

In [14]:
def process_mcid(mcid_input):
    global df_list, mcid
    mcid = mcid_input
    df_list = json.loads(df_json)
    
    analysis_crew.kickoff({"df_list": df_list, "mcid": mcid})
    
    patient_records = get_agent_result("Patient Records")
    summary = get_agent_result("Summary")
    common_diseases = get_agent_result("Common Diseases")
    
    # Format the outputs
    formatted_records = f"Patient Records for MCID {mcid}:\n\n{patient_records}"
    formatted_summary = f"Patient Summary:\n\n{summary}"
    formatted_diseases = f"Common Diseases Assessment:\n\n{common_diseases}"
    
    return formatted_records, formatted_summary, formatted_diseases

def chat_with_bot(user_question):
    if not results_store:
        return "Please enter a patient MCID first."
    
    chatbot_response = chatbot_crew.kickoff({"user_question": user_question})
    return chatbot_response


In [15]:
with gr.Blocks() as demo:
    gr.Markdown("# Medical Record Analysis and Chatbot")
    
    with gr.Tab("Patient Analysis"):
        mcid_input = gr.Textbox(label="Enter Patient MCID")
        analyze_button = gr.Button("Analyze Patient Records")
        
        with gr.Tab("Patient Records"):
            records_output = gr.Markdown()
        with gr.Tab("Summary"):
            summary_output = gr.Markdown()
        with gr.Tab("Common Diseases"):
            diseases_output = gr.Markdown()
        
        analyze_button.click(process_mcid, 
                             inputs=mcid_input, 
                             outputs=[records_output, summary_output, diseases_output])
    
    with gr.Tab("Chat with Medical Bot"):
        chat_input = gr.Textbox(label="Ask a question about the patient or any medical query")
        chat_button = gr.Button("Ask")
        chat_output = gr.Markdown()
        
        chat_button.click(chat_with_bot, inputs=chat_input, outputs=chat_output)

In [16]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://48c17ca925870f29e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 [2024-08-06 19:01:49][DEBUG]: == Working Agent: Get patient's medical records
 

I encountered an error while trying to use the tool. This was the error: patientRecordSelect_tool() missing 1 required positional argument: 'df_list'.
 Tool patientRecordSelect_tool accepts these inputs: patientRecordSelect_tool() - Select the patient's medical records for patient with given mcid.     'dos': date of service.     'cd_value': ICD10 diagnosis codes.     'ICD_FULL_DESC': description of ICD10 code.     

description="Get the medical records for the given patient 2 from file {'mcid': {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 3, '8': 3, '9': 3, '10': 3, '11': 3, '12': 3, '13': 3, '14': 3, '15': 3, '16': 3, '17': 3, '18': 3, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 0, '25': 0, '26': 0, '27': 0, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 0, '40': 0, '41': 0, '42': 0, '43': 0, '44': 0, '45': 0, '46': 0, '47'